In [2]:
# Load Libraries
import pandas as pd
import joblib
import datetime as dt
from datetime import timedelta
import numpy as np
from pathlib import Path
import os

In [3]:
# Load data from plk file
s_path=Path.cwd()
s_prt_path=s_path.parent
file=os.path.join(s_prt_path,'datasets\DataModel.plk')
DicData = joblib.load(file)
#typeAnalysis='Hourly'
typeAnalysis='Hourly'

In [4]:
# Get daily variables to csv export
df_Day=DicData['Datos_day'][['Date','VolUtil','Aportes']]

# Sort and define index
df_Day.sort_values(by='Date',inplace=True)
#df_Day.set_index('Date',inplace=True)
# Export data to csv file
file=os.path.join(s_prt_path,'datasets\XM_D.csv')
# df_Day.to_csv(file)
file=os.path.join(s_prt_path,'datasets\Dayslabels.csv')
df_typeday = pd.read_csv(file, index_col=0)
df_typeday.index = pd.to_datetime(df_typeday.index)
df_typeday.reset_index(inplace=True)

C:\Users\alejo\AppData\Local\Temp\ipykernel_10288\722487016.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_Day.sort_values(by='Date',inplace=True)


In [5]:
df_Day.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2191 entries, 0 to 2190
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     2191 non-null   datetime64[ns]
 1   VolUtil  2191 non-null   float64       
 2   Aportes  2191 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 68.5 KB


In [6]:
# Obtener infomración de los precios de oferta
df_PrecioOfe=DicData['POfe_Rec']
group_PO_day=df_PrecioOfe.groupby('Date').mean()
group_PO_day.rename(columns={'Value':'PrecioO'},inplace=True)

C:\Users\alejo\AppData\Local\Temp\ipykernel_10288\2198124498.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  group_PO_day=df_PrecioOfe.groupby('Date').mean()


In [7]:
group_PO_day.reset_index(inplace=True)
group_PO_day.head(5)


,Date,PrecioO
0,2013-01-01,349.839712
1,2013-01-02,352.772108
2,2013-01-03,354.570024
3,2013-01-04,353.987962
4,2013-01-05,352.864503


In [8]:
# Agregar las características diarias en el dataset
df_Day = df_Day.merge(df_typeday, on='Date', how='inner')[['Date','VolUtil','Aportes','day_cls']]
df_Day = df_Day.merge(group_PO_day, on='Date', how='inner')[['Date','VolUtil','Aportes','PrecioO','day_cls']]

In [9]:
# Obtener año, mes, día
df_Day.reset_index(inplace=True)
# Add Year, Month and Day
df_Day['Year']=df_Day['Date'].dt.year
df_Day['Month']=df_Day['Date'].dt.month
df_Day['Day']=df_Day['Date'].dt.day

In [10]:
df_Day.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2191 entries, 0 to 2190
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   index    2191 non-null   int64         
 1   Date     2191 non-null   datetime64[ns]
 2   VolUtil  2191 non-null   float64       
 3   Aportes  2191 non-null   float64       
 4   PrecioO  2191 non-null   float64       
 5   day_cls  2191 non-null   int64         
 6   Year     2191 non-null   int64         
 7   Month    2191 non-null   int64         
 8   Day      2191 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(5)
memory usage: 154.2 KB


In [13]:
# Generar los datos de 24 periodos a las varables diarias
df_HourAux=df_Day.copy()
for i in range(1,25):
    if i==1:
        df_HourAux['Hour']=i
    else:
        df_Aux=df_Day.copy()
        df_Aux['Hour']=i
        df_HourAux=pd.concat([df_HourAux, df_Aux],axis=0)
# df_HourAux.reset_index(inplace=True)


In [14]:
df_HourAux.head(20)

,index,Date,VolUtil,Aportes,PrecioO,day_cls,Year,Month,Day,Hour
0,0,2013-01-01,1.115503e+10,64981500.0,349.839712,0,2013,1,1,1
1,1,2013-01-02,1.110879e+10,69612400.0,352.772108,0,2013,1,2,1
2,2,2013-01-03,1.105461e+10,63955500.0,354.570024,2,2013,1,3,1
3,3,2013-01-04,1.098883e+10,56189600.0,353.987962,2,2013,1,4,1
4,4,2013-01-05,1.093476e+10,57780600.0,352.864503,2,2013,1,5,1
5,5,2013-01-06,1.088843e+10,58862700.0,343.191191,0,2013,1,6,1
6,6,2013-01-07,1.083696e+10,51303300.0,340.455816,0,2013,1,7,1
7,7,2013-01-08,1.075990e+10,46439200.0,345.420357,2,2013,1,8,1
8,8,2013-01-09,1.067779e+10,44430300.0,341.706899,1,2013,1,9,1
9,9,2013-01-10,1.060226e+10,49541500.0,352.778295,1,2013,1,10,1


In [15]:
df_HourAux.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 52584 entries, 0 to 2190
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   index    52584 non-null  int64         
 1   Date     52584 non-null  datetime64[ns]
 2   VolUtil  52584 non-null  float64       
 3   Aportes  52584 non-null  float64       
 4   PrecioO  52584 non-null  float64       
 5   day_cls  52584 non-null  int64         
 6   Year     52584 non-null  int64         
 7   Month    52584 non-null  int64         
 8   Day      52584 non-null  int64         
 9   Hour     52584 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(6)
memory usage: 4.4 MB


In [16]:
# Obtener las características de tipo horario
df_PB=DicData['PBolsa_hour']
df_Dem=DicData['DemReal_hour']
df_Disp=DicData['DispAgre_hour']
df_Disp_Ter=DicData['Disp_Ter_hour_Rec']
df_Disp_NoTer=DicData['Disp_NoTer_hour_Rec']
df_Disp_NoTer=DicData['Disp_NoTer_hour_Rec']


In [17]:
# Dar formato a la fecha con la hora
df_PB['Date']=pd.to_datetime(df_PB['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')) + pd.to_timedelta(df_PB['Hour']-1, unit='h')
df_Dem['Date']=pd.to_datetime(df_Dem['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')) + pd.to_timedelta(df_Dem['Hour']-1, unit='h')
df_Disp['Date']=pd.to_datetime(df_Disp['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')) + pd.to_timedelta(df_Disp['Hour']-1, unit='h')
df_Disp_Ter['Date']=pd.to_datetime(df_Disp_Ter['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')) + pd.to_timedelta(df_Disp_Ter['Hour']-1, unit='h')
df_Disp_NoTer['Date']=pd.to_datetime(df_Disp_NoTer['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')) + pd.to_timedelta(df_Disp_NoTer['Hour']-1, unit='h')
df_HourAux['Date']=pd.to_datetime(df_HourAux['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')) + pd.to_timedelta(df_HourAux['Hour']-1, unit='h')

In [21]:
# Merge dataframe to get all colums in one dataframe
df_DataModel = df_PB.merge(df_Dem, on='Date', how='inner')[['Date','PrecioB','Demanda']]
df_DataModel = df_DataModel.merge(df_Disp_Ter, on='Date', how='inner')[['Date','PrecioB','Demanda','DispTer']]
df_DataModel = df_DataModel.merge(df_Disp_NoTer, on='Date', how='inner')[['Date','PrecioB','Demanda','DispTer','DispNoTer']]

if typeAnalysis=='Hourly':
    df_DataModel = df_DataModel.merge(df_HourAux, on='Date', how='inner')[['Date','PrecioB','Demanda','DispTer','DispNoTer','VolUtil','Aportes','PrecioO','day_cls']]

In [22]:
# Sort dataframe by date and set the index
df_DataModel=df_DataModel.sort_values(by='Date')
df_DataModel.set_index('Date',inplace=True)
df_DataModel

,PrecioB,Demanda,DispTer,DispNoTer,VolUtil,Aportes,PrecioO,day_cls
Date,,,,,,,,
2013-01-01 00:00:00,168.39165,5635708.86,4312666.0,8609000.0,1.115503e+10,64981500.0,349.839712,0
2013-01-01 01:00:00,168.39165,5398892.95,4245000.0,8609000.0,1.115503e+10,64981500.0,349.839712,0
2013-01-01 02:00:00,170.55065,5132168.81,4245000.0,8609000.0,1.115503e+10,64981500.0,349.839712,0
2013-01-01 03:00:00,150.55165,4886320.69,4241500.0,8609000.0,1.115503e+10,64981500.0,349.839712,0
2013-01-01 04:00:00,147.01865,4735282.07,4222000.0,8609000.0,1.115503e+10,64981500.0,349.839712,0
...,...,...,...,...,...,...,...,...
2018-12-31 19:00:00,383.92114,8283721.31,4717500.0,9372000.0,1.216072e+10,66390600.0,349.531895,0
2018-12-31 20:00:00,383.92114,7909574.43,4629000.0,9372000.0,1.216072e+10,66390600.0,349.531895,0
2018-12-31 21:00:00,383.92114,7382708.47,4629000.0,8834000.0,1.216072e+10,66390600.0,349.531895,0


In [23]:
# Export data to csv file
file=os.path.join(s_prt_path,'datasets\XM_H.csv')
df_DataModel.to_csv(file)